In [15]:
!pip install google-generativeai -q

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflowjs 3.18.0 requires packaging~=20.9, but you have packaging 24.2 which is incompatible.


In [59]:
import pandas as pd
import chromadb
import os
import pymongo

from chromadb.utils import embedding_functions

from dotenv import load_dotenv

from google import genai
from google.genai import types
from tqdm import tqdm

import time

print(load_dotenv())
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../credentials/ai_platform.json'

True


In [ ]:
db = pymongo.MongoClient(os.getenv("MONGO_URI"))["hireonai"]
jobs_collection = db.get_collection('jobs')
cursor = jobs_collection.find({})  # Query all documents
jobs_df = pd.DataFrame(list(cursor))[['_id', 'jobPosition', 'jobDescList', 'jobQualificationsList']]
jobs_df.head()

,_id,jobPosition,jobDescList,jobQualificationsList
0,68341f06d64eecb3953d58ee,Back End Developer,"[Utilize languages like Go, PHP/Laravel, and P...",[Minimum Bachelor of Science in computer scien...
1,68341f06d64eecb3953d58ef,(Freelance) Sales Retail Mobile/Canvasser,[Retail Sales Mobile akan bertanggung jawab un...,"[Pendidikan minimal SMA/SMK/sederajat., Memili..."
2,68341f06d64eecb3953d58f0,Creative Lead,"[Membuat laporan weekly & monthly, Memberikan ...",[Pendidikan minimal S1 Ilmu Komunikasi / Marke...
3,68341f06d64eecb3953d58f1,Social Media Specialist,[We are looking for a Social Media Specialist ...,"[Bachelor's degree in Marketing, Communication..."
4,68341f06d64eecb3953d58f2,HRGA Staff area Padalarang Kabupaten Bandung B...,[Membantu perekrutan dan pengelolaan operasion...,"[Pengalaman HRGA minimal 1 Tahun, Memiliki kem..."


In [30]:
all_id_list = jobs_df['_id'].astype(str).tolist()
all_job_title_list = jobs_df['jobPosition'].tolist()
all_job_desc_qual_list = jobs_df['jobDescListFormatted'].tolist()
print(len(all_id_list), len(all_job_title_list), len(all_job_desc_qual_list))
print(all_id_list[:5])
print(all_job_title_list[:5])
print(all_job_desc_qual_list[:5])


2697 2697 2697
['68341f06d64eecb3953d58ee', '68341f06d64eecb3953d58ef', '68341f06d64eecb3953d58f0', '68341f06d64eecb3953d58f1', '68341f06d64eecb3953d58f2']
['Back End Developer', '(Freelance) Sales Retail Mobile/Canvasser', 'Creative Lead', 'Social Media Specialist', 'HRGA Staff area Padalarang Kabupaten Bandung Barat']
['Job Desc:\n- Utilize languages like Go, PHP/Laravel, and Python for backend development and maintenance.\n- Manage SQL (MySQL, PostgreSQL) and NoSQL (MongoDB) databases for data storage and retrieval.\n- Implement and manage messaging systems such as Kafka, RabbitMQ, and Redis for enhanced performance.\n- Design and implement APIs for effective frontend-backend communication.\n- Regularly use Git for code versioning and collaborative development.\n- Apply clean code architecture practices to ensure code quality and maintainability.\n- Use object-oriented programming techniques for modular and scalable code development.\n- Understand and implement Entity Relationship D

In [9]:
chroma_client = chromadb.HttpClient(host=os.getenv("CHROMA_CLIENT_HOST"), port=8000)

In [55]:
client = genai.Client()
response = client.models.embed_content(
            model="text-multilingual-embedding-002",
            contents=all_job_title_list[0],  # Send only one item
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_DOCUMENT",
                title='Job Title',
            ),
        )
        # Extract embedding from response and add to list
embedding = response.embeddings[0].values

In [60]:
# Create empty list to store job_title_embeddings
job_title_embeddings = []

# Split job titles into batches of 250
batch_size = 250
batches = [all_job_title_list[i:i + batch_size] for i in range(0, len(all_job_title_list), batch_size)]

# Counter for processed items
processed_count = 0

# Process batches with progress bar
for batch_idx, batch in enumerate(tqdm(batches, desc="Processing batches")):
    try:
        response = client.models.embed_content(
            model="text-multilingual-embedding-002",
            contents=batch,  # Send batch of items
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_DOCUMENT",
                title='Job Title',
            ),
        )
        
        # Extract embeddings from response and add to list
        batch_embeddings = [embedding.values for embedding in response.embeddings]
        job_title_embeddings.extend(batch_embeddings)
        processed_count += len(batch)
        
    except Exception as e:
        print(f"Error processing batch: {e}")
        # Fall back to processing one by one if batch fails
        for job_title in batch:
            try:
                single_response = client.models.embed_content(
                    model="text-multilingual-embedding-002",
                    contents=job_title,
                    config=types.EmbedContentConfig(
                        task_type="RETRIEVAL_DOCUMENT",
                        title='Job Title',
                    ),
                )
                embedding = single_response.embeddings[0].values
                job_title_embeddings.append(embedding)
                processed_count += 1
            except Exception as inner_e:
                print(f"Error processing '{job_title}': {inner_e}")
    
    # Check if we need to add a delay (after every 1500 items)
    if processed_count >= 1500:
        delay_minutes = 2
        print(f"\nProcessed {processed_count} items. Adding {delay_minutes} minute delay...")
        time.sleep(delay_minutes * 60)  # Convert minutes to seconds
        processed_count = processed_count % 1500  # Keep only the remainder

print(f"Generated {len(job_title_embeddings)} embeddings")

Processing batches:  45%|████▌     | 5/11 [00:17<00:20,  3.49s/it]


Processed 1500 items. Adding 2 minute delay...


Processing batches: 100%|██████████| 11/11 [02:40<00:00, 14.55s/it]

Generated 2697 embeddings


In [66]:
chroma_client.delete_collection('job_titles') if chroma_client.get_collection('job_titles') else None

In [68]:
# Batch size for adding to collection
batch_size = 100

# Split all data into manageable batches
total_items = len(all_id_list)
batches = [(i, min(i + batch_size, total_items)) for i in range(0, total_items, batch_size)]

# Create or get a collection
collection = chroma_client.get_or_create_collection(
    name="job_titles",
    embedding_function=embedding_functions.GoogleGenerativeAiEmbeddingFunction(
        model_name="text-multilingual-embedding-002",
        task_type="RETRIEVAL_DOCUMENT",
        api_key=os.getenv("GEMINI_API_KEY")
    )
)

# Add documents in batches
for start_idx, end_idx in tqdm(batches, desc="Adding to Chroma collection"):
    batch_ids = [str(id) for id in all_id_list[start_idx:end_idx]]
    batch_documents = all_job_title_list[start_idx:end_idx]
    batch_embeddings = job_title_embeddings[start_idx:end_idx]
    
    try:
        collection.add(
            ids=batch_ids,
            documents=batch_documents,
            embeddings=batch_embeddings
        )
    except Exception as e:
        print(f"Error adding batch {start_idx}:{end_idx}: {e}")

print(f"Added data to collection in {len(batches)} batches")
print(f"Collection count: {collection.count()}")

Adding to Chroma collection: 100%|██████████| 27/27 [01:04<00:00,  2.39s/it]


Added data to collection in 27 batches
Collection count: 2697


In [72]:
client = genai.Client()
query_embeddings = client.models.embed_content(
    model="text-multilingual-embedding-002",
    contents=["software developer"],
    config=types.EmbedContentConfig(
        task_type="RETRIEVAL_DOCUMENT",
        title='Job Title',
    )).embeddings[0].values

In [73]:
# 3. Example query
results = collection.query(
    query_embeddings=[query_embeddings],
    n_results=5
)

In [80]:
delay_minutes = 1.5

time.sleep(delay_minutes * 60)  # Convert minutes to seconds# Create empty list to store job_desc_req_embeddings

job_desc_req_embeddings = []

# Split job titles into batches of 250
batch_size = 64
batches = [all_job_desc_qual_list[i:i + batch_size] for i in range(0, len(all_job_desc_qual_list), batch_size)]

# Counter for processed items
processed_count = 0

# Process batches with progress bar
for batch_idx, batch in enumerate(tqdm(batches, desc="Processing batches")):
    try:
        response = client.models.embed_content(
            model="text-multilingual-embedding-002",
            contents=batch,  # Send batch of items
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_DOCUMENT",
                title='Job Description and Qualification',
            ),
        )
        
        # Extract embeddings from response and add to list
        batch_embeddings = [embedding.values for embedding in response.embeddings]
        job_desc_req_embeddings.extend(batch_embeddings)
        processed_count += len(batch)
        
    except Exception as e:
        print(f"Error processing batch: {e}")
        # Fall back to processing one by one if batch fails
        for job_title in batch:
            try:
                single_response = client.models.embed_content(
                    model="text-multilingual-embedding-002",
                    contents=job_title,
                    config=types.EmbedContentConfig(
                        task_type="RETRIEVAL_DOCUMENT",
                        title='Job Title',
                    ),
                )
                embedding = single_response.embeddings[0].values
                job_desc_req_embeddings.append(embedding)
                processed_count += 1
            except Exception as inner_e:
                print(f"Error processing '{job_title}': {inner_e}")
    
    # Check if we need to add a delay (after every 1500 items)
    if processed_count >= 300:
        delay_minutes = 1.5
        print(f"\nProcessed {processed_count} items. Adding {delay_minutes} minute delay...")
        time.sleep(delay_minutes * 60)  # Convert minutes to seconds
        processed_count = processed_count % 300  # Keep only the remainder

print(f"Generated {len(job_desc_req_embeddings)} embeddings")

Processing batches:   9%|▉         | 4/43 [00:09<01:27,  2.26s/it]


Processed 320 items. Adding 1.5 minute delay...


Processing batches:  21%|██        | 9/43 [01:51<05:05,  8.97s/it]


Processed 340 items. Adding 1.5 minute delay...


Processing batches:  33%|███▎      | 14/43 [03:33<04:49,  9.99s/it]

...


Processed 312 items. Adding 1.5 minute delay...


Processing batches:  86%|████████▌ | 37/43 [12:00<01:04, 10.71s/it]


Processed 332 items. Adding 1.5 minute delay...


Processing batches: 100%|██████████| 43/43 [13:48<00:00, 19.27s/it]

Generated 2697 embeddings


In [92]:
try:
    chroma_client.delete_collection('job_desc_req')
except:
    pass

# Batch size for adding to collection
batch_size = 100

# Split all data into manageable batches
total_items = len(all_id_list)
batches = [(i, min(i + batch_size, total_items)) for i in range(0, total_items, batch_size)]

# Create or get a collection
collection = chroma_client.get_or_create_collection(
    name="job_desc_req",
    embedding_function=embedding_functions.GoogleGenerativeAiEmbeddingFunction(
        model_name="text-multilingual-embedding-002",
        task_type="RETRIEVAL_DOCUMENT",
        api_key=os.getenv("GEMINI_API_KEY")
    )
)

# Add documents in batches
for start_idx, end_idx in tqdm(batches, desc="Adding to Chroma collection"):
    batch_ids = [str(id) for id in all_id_list[start_idx:end_idx]]
    batch_documents = all_job_title_list[start_idx:end_idx]
    batch_embeddings = job_title_embeddings[start_idx:end_idx]
    
    try:
        collection.add(
            ids=batch_ids,
            documents=batch_documents,
            embeddings=batch_embeddings
        )
    except Exception as e:
        print(f"Error adding batch {start_idx}:{end_idx}: {e}")

print(f"Added data to collection in {len(batches)} batches")
print(f"Collection count: {collection.count()}")

Adding to Chroma collection: 100%|██████████| 27/27 [00:55<00:00,  2.05s/it]


Added data to collection in 27 batches
Collection count: 2697


In [93]:
chroma_client.list_collections()

[Collection(name=job_desc_req),
 Collection(name=try_gemini_embedding),
 Collection(name=job_titles)]